In [38]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_crud import AnimalShelter

# Determines number of documents to be displayed on one page of datatable
sz_page = 10

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "4444"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Use Salvare logo
image_filename = 'Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#MAIN APP LAYOUT
app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(
        html.A(
            href="https://www.snhu.edu",
            children=[
                html.Img(alt="Salvare logo", src='data:image/png;base64,{}'.format(encoded_image.decode()))
            ]
        )
    ),
    html.Center(
        html.H2("The Damean Database Datatable strikes again!")
    ),
    html.Hr(),
    html.Div(children=[
        html.Label('Filters'),
        dcc.Dropdown(id="filter-type", options=['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'No Filter'], value='No Filter')
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",
                         selected_rows=[0],
                         sort_action="native",
                         sort_mode="multi",
                         page_size=sz_page,
                         filter_action="native"
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        ## Better to use match-case here, but we are using an ancient (3.9) version of Python
        if filter_type == "Water Rescue":
            df = pd.DataFrame.from_records(db.read({
                "age_upon_outcome_in_weeks" : {"$gte": 26, "$lte": 156},
                "sex_upon_outcome" : "Intact Female",
                "$or" : [
                    {"breed" : "Labrador Retriever Mix"}, 
                    {"breed" : "Chesapeake Bay Retriever"}, 
                    {"breed" : "Newfoundland"}
                ]
            }))
        elif filter_type == "Mountain Rescue":
            df = pd.DataFrame.from_records(db.read({
                "age_upon_outcome_in_weeks" : {"$gte": 26, "$lte": 156},
                "sex_upon_outcome" : "Intact Male",
                "$or" : [
                    {"breed" : "German Shepherd"}, 
                    {"breed" : "Alaskan Malamute"}, 
                    {"breed" : "Old English Sheepdog"},
                    {"breed" : "Siberian Husky"},
                    {"breed" : "Rottweiler"}
                ]
            }))
        elif filter_type == "Disaster Rescue":
            df = pd.DataFrame.from_records(db.read({
                "age_upon_outcome_in_weeks" : {"$gte": 20, "$lte": 300},
                "sex_upon_outcome" : "Intact Male",
                "$or" : [
                    {"breed" : "German Shepherd"}, 
                    {"breed" : "Doberman Pinscher"}, 
                    {"breed" : "Golden Retriever"},
                    {"breed" : "Bloodhound"},
                    {"breed" : "Rottweiler"}
                ]
            }))
        elif filter_type == "No Filter":
            df = pd.DataFrame.from_records(db.read({}))
        else:
            df = pd.DataFrame.from_records(db.read({}))
                
        df.drop(columns=['_id'], inplace=True)
        # Table appears to only be expecting a "data" argument as it fails when returning the data and columns tuple
 #       columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')       

        return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    
    # Show a pie chart with the data of breed percentages on the current table
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    fig.update_traces(textposition='inside')
    return [
        dcc.Graph(  
            id='Animal Breeds',
            figure=fig
        )   
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    # Relevant column data is abstracted
    col_breed = 4
    col_name = 9
    col_lat = 13
    col_long = 14
    # Center Coordinates are for Austin, Tx
    au_lat = 30.75
    au_long = -97.48
        
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
        # Return a map with no marker until the derived_virt_sel_row is updated with a not None value
        return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[au_lat, au_long], zoom=10)
        ]
    else: 
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[au_lat, au_long], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  dl.Marker(position=[dff.iloc[row,col_lat], dff.iloc[row,col_long]],
                           children=[
                               dl.Tooltip(dff.iloc[row,col_breed]),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(dff.iloc[row, col_name])
                               ])
                           ])
              ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:13598/
